# 1. Imports

In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np

# 2. Importing the Datasets

In [2]:
ddos_portmap_2018_df = pd.read_csv("../data/train_test/CSE_CIC_IDS2018/ddos_portmap_2018_train.csv", index_col=0)

In [3]:
ddos_ldap_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_ldap_2019_train.csv", index_col=0)

In [4]:
ddos_netbios_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_netbios_2019_train.csv", index_col=0)

In [5]:
ddos_syn_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_syn_2019_train.csv", index_col=0)

In [6]:
ddos_udp_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_udp_2019_train.csv", index_col=0)

In [2]:
target_feature = 'Label'

# 3. Feature Selection Method

In [8]:
'''from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def forward_selection(X, y, features, metric='accuracy'):
    selected_features = []
    selected_scores = []

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    while len(selected_features) < len(features):
        remaining_features = list(set(features) - set(selected_features))

        if not remaining_features:
            break  # Exit the loop if there are no more features to evaluate

        best_score = 0
        best_feature = None

        for feature in remaining_features:
            current_features = selected_features + [feature]

            model = RandomForestClassifier(random_state=42)
            model.fit(X_train[current_features], y_train)

            y_pred = model.predict(X_test[current_features])

            if metric == 'accuracy':
                score = accuracy_score(y_test, y_pred)
                print(f"Feature: {feature}, Score: {score}")

            if score > best_score:
                best_score = score
                best_feature = feature

        selected_features.append(best_feature)
        selected_scores.append(best_score)
        print(f"Selected Features: {selected_features}")

    return selected_features, selected_scores'''




'from sklearn.model_selection import train_test_split\nfrom sklearn.metrics import accuracy_score\nfrom sklearn.ensemble import RandomForestClassifier\nimport numpy as np\n\ndef forward_selection(X, y, features, metric=\'accuracy\'):\n    selected_features = []\n    selected_scores = []\n\n    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n    while len(selected_features) < len(features):\n        remaining_features = list(set(features) - set(selected_features))\n\n        if not remaining_features:\n            break  # Exit the loop if there are no more features to evaluate\n\n        best_score = 0\n        best_feature = None\n\n        for feature in remaining_features:\n            current_features = selected_features + [feature]\n\n            model = RandomForestClassifier(random_state=42)\n            model.fit(X_train[current_features], y_train)\n\n            y_pred = model.predict(X_test[current_features])\n\n            if metr

In [9]:
'''def stable_ForwardSelection(train_df, num_splits, verbose):
    selected_features = []

    df_split = np.array_split(train_df, num_splits)

    for small_df in df_split:
        X_train = small_df.drop(['Label'], axis=1)
        y_train = small_df['Label']


        features, _ = forward_selection(X_train, y_train, X_train.columns)
        selected_features += features

    # Use forward selection on the entire training dataset
    forward_selected_features, _ = forward_selection(train_df.drop(['Label'], axis=1), train_df['Label'], train_df.columns)

    # Extract the selected features from the tuple returned by forward_selection
    selected_features += forward_selected_features

    rank_data = {'feature': train_df.columns.tolist(),
                 'forward_selection_rank': [selected_features.count(feature) for feature in train_df.columns]}

    rank_df = pd.DataFrame(rank_data)

    if verbose:
        print(rank_df)

    return rank_df
'''



"def stable_ForwardSelection(train_df, num_splits, verbose):\n    selected_features = []\n\n    df_split = np.array_split(train_df, num_splits)\n\n    for small_df in df_split:\n        X_train = small_df.drop(['Label'], axis=1)\n        y_train = small_df['Label']\n\n\n        features, _ = forward_selection(X_train, y_train, X_train.columns)\n        selected_features += features\n\n    # Use forward selection on the entire training dataset\n    forward_selected_features, _ = forward_selection(train_df.drop(['Label'], axis=1), train_df['Label'], train_df.columns)\n\n    # Extract the selected features from the tuple returned by forward_selection\n    selected_features += forward_selected_features\n\n    rank_data = {'feature': train_df.columns.tolist(),\n                 'forward_selection_rank': [selected_features.count(feature) for feature in train_df.columns]}\n\n    rank_df = pd.DataFrame(rank_data)\n\n    if verbose:\n        print(rank_df)\n\n    return rank_df\n"

In [3]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

def rfe_selection(X_train, y_train, n_features_to_select):
    
    features =  X_train.columns
    
    lreg = LinearRegression()
    
    # Create an RFE object for feature selection
    rfe = RFE(lreg, n_features_to_select=n_features_to_select)
    
    # Fit the RFE to the training data
    rfe.fit(X_train, y_train)
    
    # Get the selected feature ranking
    feature_ranking = rfe.ranking_

    # Get the selected feature names based on their ranking
    selected_features = list(features[np.argsort(feature_ranking)])
    
    return selected_features[0:n_features_to_select]

In [4]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

def stable_RFE(train_df, num_splits, n_features_to_select, verbose):
    selectedFeatures = []

    df_split = np.array_split(train_df, num_splits)

    for small_df in df_split:
        X_train = small_df.drop(['Label'], axis=1)
        y_train = small_df['Label']

        features = rfe_selection(X_train, y_train, n_features_to_select)

        selectedFeatures = selectedFeatures + features
       
        
    rank_data = {}
    features = train_df.columns.tolist()
    ranks = []
        
    for feature in features:
        if verbose :
            print("Feature: "+feature+". Count: "+str(selectedFeatures.count(feature))+"/"+str(num_splits))
        ranks.append(selectedFeatures.count(feature)/num_splits)
    
    rank_data = {'feature':features, 'rank':ranks}
    rank_df = pd.DataFrame(rank_data) 
    
    return rank_df

In [22]:
portmap_ranking = stable_RFE(ddos_portmap_2018_df, 10, 50, verbose=False)

In [23]:
portmap_ranking.head(20)

,feature,rank
0,DestinationPort,1.0
1,FlowDuration,1.0
2,TotalFwdPackets,1.0
3,TotalBackwardPackets,1.0
4,TotalLengthofFwdPackets,1.0
5,TotalLengthofBwdPackets,1.0
6,FwdPacketLengthMax,1.0
7,FwdPacketLengthMin,1.0
8,FwdPacketLengthMean,1.0
9,FwdPacketLengthStd,1.0


# 4. Saving the Ranking

In [28]:
portmap_ranking = stable_RFE(ddos_portmap_2018_df, 10, 50, verbose=False)
portmap_ranking.to_csv("../ranking/CSE_CIC_IDS2018/ddos_portmap_2018_RFE.csv")
portmap_ranking.head(20)

,feature,rank
0,DestinationPort,1.0
1,FlowDuration,0.6
2,TotalFwdPackets,1.0
3,TotalBackwardPackets,1.0
4,TotalLengthofFwdPackets,0.8
5,TotalLengthofBwdPackets,1.0
6,FwdPacketLengthMax,1.0
7,FwdPacketLengthMin,1.0
8,FwdPacketLengthMean,1.0
9,FwdPacketLengthStd,1.0


In [29]:
ldap_ranking = stable_RFE(ddos_ldap_2019_df, 10, 50, verbose=False)
ldap_ranking.to_csv("../ranking/CIC_DDoS2019/ddos_ldap_2019_RFE.csv")
ldap_ranking.head(20)

,feature,rank
0,FlowID,0.0
1,SourceIP,1.0
2,SourcePort,0.7
3,DestinationIP,0.4
4,DestinationPort,0.0
5,Protocol,1.0
6,FlowDuration,0.7
7,TotalFwdPackets,1.0
8,TotalBackwardPackets,0.9
9,TotalLengthofFwdPackets,1.0


In [30]:
netbios_ranking = stable_RFE(ddos_netbios_2019_df, 10, 50, verbose=False)
netbios_ranking.to_csv("../ranking/CIC_DDoS2019/ddos_netbios_2019_RFE.csv")
netbios_ranking.head(20)

,feature,rank
0,FlowID,0.0
1,SourceIP,0.0
2,SourcePort,0.0
3,DestinationIP,0.0
4,DestinationPort,0.0
5,Protocol,0.2
6,FlowDuration,1.0
7,TotalFwdPackets,1.0
8,TotalBackwardPackets,1.0
9,TotalLengthofFwdPackets,1.0


In [31]:
syn_ranking = stable_RFE(ddos_syn_2019_df, 10, 50, verbose=False)
syn_ranking.to_csv("../ranking/CIC_DDoS2019/ddos_syn_2019_RFE.csv")
syn_ranking.head(20)

,feature,rank
0,FlowID,0.0
1,SourceIP,0.2
2,SourcePort,0.0
3,DestinationIP,0.1
4,DestinationPort,0.0
5,Protocol,1.0
6,FlowDuration,1.0
7,TotalFwdPackets,1.0
8,TotalBackwardPackets,1.0
9,TotalLengthofFwdPackets,1.0


In [27]:
udp_ranking = stable_RFE(ddos_udp_2019_df, 10, 50, verbose=False)
udp_ranking.to_csv("../ranking/CIC_DDoS2019/ddos_udp_2019_RFE.csv")
udp_ranking.head(20)

,feature,rank
0,FlowID,0.0
1,SourceIP,0.2
2,SourcePort,0.0
3,DestinationIP,0.0
4,DestinationPort,0.0
5,Protocol,0.8
6,FlowDuration,1.0
7,TotalFwdPackets,1.0
8,TotalBackwardPackets,1.0
9,TotalLengthofFwdPackets,0.7


# 5. Baselines

In [5]:
def baseline_RFE(train_df, n_features_to_select, verbose):
    
    selectedFeatures = []
    
    X_train = train_df.drop([target_feature], axis=1)
    y_train = train_df[target_feature]
        
    selectedFeatures =  rfe_selection(X_train, y_train, n_features_to_select)
        
    rank_data = {}
    features = train_df.columns.tolist()
    ranks = []
        
    for feature in features:
        if verbose :
            print("Feature: "+feature+". Count: "+str(selectedFeatures.count(feature)))
        ranks.append(selectedFeatures.count(feature))
    
    rank_data = {'feature':features, 'rank':ranks}
    rank_df = pd.DataFrame(rank_data)
    
    return rank_df

In [33]:
portmap_ranking_baseline = baseline_RFE(ddos_portmap_2018_df, 50, verbose=False)
portmap_ranking_baseline.to_csv("../baseline_ranking/CSE_CIC_IDS2018/ddos_portmap_2018_RFE.csv")
portmap_ranking_baseline.head(5)

,feature,rank
0,DestinationPort,1
1,FlowDuration,0
2,TotalFwdPackets,1
3,TotalBackwardPackets,1
4,TotalLengthofFwdPackets,1


In [34]:
ldap_ranking_baseline = baseline_RFE(ddos_ldap_2019_df, 50, verbose=False)
ldap_ranking_baseline.to_csv("../baseline_ranking/CIC_DDoS2019/ddos_ldap_2019_RFE.csv")
ldap_ranking_baseline.head(5)

,feature,rank
0,FlowID,0
1,SourceIP,1
2,SourcePort,1
3,DestinationIP,1
4,DestinationPort,0


In [35]:
udp_ranking_baseline = baseline_RFE(ddos_udp_2019_df, 50, verbose=False)
udp_ranking_baseline.to_csv("../baseline_ranking/CIC_DDoS2019/ddos_udp_2019_RFE.csv")
udp_ranking_baseline.head(5)

,feature,rank
0,FlowID,0
1,SourceIP,1
2,SourcePort,0
3,DestinationIP,1
4,DestinationPort,0


In [36]:
netbios_ranking_baseline = baseline_RFE(ddos_netbios_2019_df, 50, verbose=False)
netbios_ranking_baseline.to_csv("../baseline_ranking/CIC_DDoS2019/ddos_netbios_2019_RFE.csv")
netbios_ranking_baseline.head(5)

,feature,rank
0,FlowID,0
1,SourceIP,1
2,SourcePort,0
3,DestinationIP,1
4,DestinationPort,0


In [37]:
syn_ranking_baseline = baseline_RFE(ddos_syn_2019_df, 50, verbose=False)
syn_ranking_baseline.to_csv("../baseline_ranking/CIC_DDoS2019/ddos_syn_2019_RFE.csv")
syn_ranking_baseline.head(5)

,feature,rank
0,FlowID,0
1,SourceIP,1
2,SourcePort,0
3,DestinationIP,1
4,DestinationPort,0


# 6. Repeating with Assignment Datasets

In [6]:
a2_train = pd.read_csv("../data/csi5388_assignment2_3_data/assignment2_train.csv", index_col=0)
a3_train = pd.read_csv("../data/csi5388_assignment2_3_data/assignment3_train.csv", index_col=0)

a2ranking = stable_RFE(a2_train, 10, 25, verbose=False)
a2ranking.to_csv("../ranking/csi5388_assignment2_3_data/assignment2_RFE.csv")
a2ranking.head(20)

a3ranking = stable_RFE(a3_train, 10, 4, verbose=False)
a3ranking.to_csv("../ranking/csi5388_assignment2_3_data/assignment3_RFE.csv")
a3ranking.head(10)

# Baselines 

a2_baseline = baseline_RFE(a2_train, 25, verbose=False)
a2_baseline.to_csv("../baseline_ranking/csi5388_assignment2_3_data/assignment2_RFE.csv")
a2_baseline.head(5)

a3_baseline = baseline_RFE(a3_train, 4, verbose=False)
a3_baseline.to_csv("../baseline_ranking/csi5388_assignment2_3_data/assignment3_RFE.csv")
a3_baseline.head(5)

,feature,rank
0,length_longest_word,0
1,FQDN_count,0
2,sld_vec,0
3,private_192,1
4,lower,0
